## mnist-hpo
This notebook uses Keras to build a model for the mnist dataset with the network's learning rate configurable as a hyperparameter in order to publish in Watson Studios' Deep Learning as a Service

First, libraries, including Keras, are imported

In [1]:
import os
import json
import keras
import os.path
from os import environ
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout

Using TensorFlow backend.


Watson Studio will send a json for the Keras model with information on how to vary the models' learning rate. More information on how this is performed is in the Guide of the DLaaSWorkshop repo

In [2]:
###############################################################################
# Set up HPO.
###############################################################################

config_file = "config.json"

if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        json_obj = json.load(f)
    learning_rate = json_obj["learning_rate"]
else:
    learning_rate = 0.001


###############################################################################

In [3]:
#loading the dataset, setting parameters
batch_size = 128
num_classes = 10
epochs = 1

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 2s 0us/step
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices, 
# each class has a node in the last layer of the network, which calculates the strength of a prediction for each class
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

5
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


In [ ]:
# model building with Keras

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [6]:
# Code to execute Keras training, observe model statistics
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['accuracy'])


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))



print("Training history:" + str(history.history))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 30s 501us/step - loss: 0.2469